### Preparación de datos

In [3]:
from pathlib import Path
import pandas as pd

# === RUTAS ===

ruta_acta =  Path(r"20251003/20251003/ADP_DTM_FACT.Acta.csv")
ruta_insumo =  Path(r"20251003/20251003/ADP_DTM_DIM.Insumo.csv")
ruta_proyectos =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")
ruta_capitulos = Path(r"20251003/20251003/ADP_DTM_DIM.CapituloPresupuesto.csv")
ruta_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")




In [7]:
# === CARGA ===
tabla_proyeccion = pd.read_csv(ruta_proyeccion)
tabla_items = pd.read_csv(ruta_items)
tabla_proyectos = pd.read_csv(ruta_proyectos)
tabla_capitulos = pd.read_csv(ruta_capitulos)
tabla_insumos = pd.read_csv(ruta_insumo)

C:\Users\aleja\AppData\Local\Temp\ipykernel_30052\1973435843.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  tabla_items = pd.read_csv(ruta_items)


In [8]:
tabla_base = tabla_proyeccion.copy()

# Merges previos
tabla_1 = pd.merge(tabla_base, tabla_proyectos, on="SkIdProyecto", how="left")
tabla_2 = pd.merge(tabla_1, tabla_capitulos, on="SkIdCapitulo", how="left")

# Asegura unicidad en Items
tabla_items_unica = tabla_items.drop_duplicates(subset=["SkIdItems"], keep="first")
tabla_3 = pd.merge(
    tabla_2, tabla_items_unica,
    on="SkIdItems", how="left",
    suffixes=("", "_item")   # control de sufijos para items
)

# 🔧 FIX insumos: prefijar y garantizar clave única
tabla_insumos_unica = tabla_insumos.drop_duplicates(subset=["SkIdInsumo"], keep="first").copy()
cols_no_clave_insumo = [c for c in tabla_insumos_unica.columns if c != "SkIdInsumo"]
tabla_insumos_pref = tabla_insumos_unica.rename(
    columns={c: f"Insumo_{c}" for c in cols_no_clave_insumo}
)

# (Opcional) Debug rápido de posibles overlaps antes del merge final
overlap_prev = set(tabla_3.columns).intersection(set(tabla_insumos_pref.columns))
print(f"[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {overlap_prev}")

# Merge final (sin sufijos porque ya prefijamos)
tabla_4 = pd.merge(
    tabla_3, tabla_insumos_pref,
    on="SkIdInsumo", how="left",
    validate="many_to_one"
)

print(f"[DEBUG] OK. Forma final: {tabla_4.shape}")

tabla_4.to_csv(
    "tabla_looker.csv",
    index=False,
    encoding="utf-8",
    sep=","                
)

[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {'SkIdInsumo'}
[DEBUG] OK. Forma final: (273450, 107)


In [ ]:
# === SELECCIÓN DE COLUMNAS ===
columnas_finales = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "Nombre Proyecto", "Capitulo Descripcion", "Item Descripcion", "Insumo_Insumo Descripcion", "Insumo_Agrupacion Descripcion"
    "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total", "Insumo_Valor Unitario", "Insumo_Valor Neto", "Insumo_Fecha Creacion",
    "Cantidad Item", "Macroproyecto Descripcion", "Insumo_Fecha Modificacion",
    "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalización", "SkIdFecha", "Capitulo Numero", "Cantidad_Item"
]

# Filtrar solo las columnas que existan realmente (por seguridad)
columnas_existentes = [col for col in columnas_finales if col in tabla_4.columns]
tabla_looker = tabla_4[columnas_existentes].copy()

NameError: name 'tabla_final' is not defined

#### Debug